# Use Case 3: Auckland Urban Infrastructure Mapping
Urban planning and transportation analysis requires comprehensive spatial data covering built environments and mobility networks. Using OpenStreetMap's community-contributed data, we'll visualise **Auckland's road network and building footprints**, with roads colour-coded by maximum speed limits to reveal the city's transportation hierarchy and urban density patterns.

## Key Technical Highlights
- **Dataset Scale:** Complete OSM road network, footpaths, and building polygons for Auckland region
- **Performance:** GPU-accelerated rendering of complex urban geometries and speed classifications
- **Interactivity:** Real-time exploration of transportation networks with speed-based styling
- **Memory Efficiency:** PyArrow's efficient handling of mixed geometry types and categorical attributes

## What You'll See
This demonstration showcases Auckland's urban fabric through its transportation infrastructure and built environment, revealing how speed limits define functional road hierarchies while building densities illustrate urban development patterns through high-performance geospatial visualisation.

## Datasets

### OpenStreetMap: Auckland Urban Infrastructure
Our urban data comes from **OpenStreetMap (OSM)** - the world's largest collaborative mapping project:

<img src="images/osm.gif" alt="drawing" width="1200"/>

- **Coverage**: Complete Auckland region including urban core and suburban areas
- **Data Currency**: Community-maintained with regular updates from local contributors
- **Licensing**: Open Database License (ODbL) enabling free use and redistribution
- **Data Quality**: Crowd-sourced verification with high accuracy in urban areas

In [1]:
from lonboard import Map, PathLayer, PolygonLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_8
from pyarrow.compute import divide, drop_null
from pyarrow.compute import max as pc_max
from pyarrow.compute import min as pc_min
from pyarrow.parquet import read_table
from arro3.io import read_parquet

In [2]:
# Read the data
roads = read_table("data/auckland-osm-roads.parquet")
buildings = read_parquet("data/auckland-osm-buildings.parquet").read_all()

In [3]:
# Set how we want to colour the map
normalised_speeds = roads["maxspeed"].to_numpy() / float(pc_max(roads["maxspeed"]))
colour_map = apply_continuous_cmap(normalised_speeds, Oranges_8)

In [4]:
road_layer = PathLayer(
    roads,
    get_color=colour_map,
    width_min_pixels=2,
    pickable=True,
)

building_layer = PolygonLayer(
    buildings,
    get_line_width=0,
    get_fill_color=[11, 41, 72],
    pickable=True,
)

In [5]:
m = Map(
    [road_layer, building_layer],
    show_tooltip=True,
    show_side_panel=True
)
m